<a href="https://colab.research.google.com/github/bingsen-wang/PowerElectronics/blob/main/PowerSwitchingDevices_MOSFET_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b>V-I Characteristics of Power MOSFET
</h1></b>

Dr. Bingsen Wang

1/1/2024
</center>

#$V_{DS}$-$I_D$ Characteristics: Ohmic Region

In the ohmic region, the following condition holds
$$\begin{align}
  v_{GS} &\gt V_{th}\\
  v_{DS} &\lt v_{GS} - V_{th}
\end{align}\tag{1}$$
where $V_{th}$ is the gate-source threshold voltage.

In ohmic region, the drain current $i_D$ is
$$ i_D =  K[2(v_{GS}-V_{th})v_{DS} - v_{DS}^2] \tag{2}$$
where $K$ is a constant determined by the physical parameters of the device.

#$V_{DS}$-$I_D$ Characteristics: Active Region

In the active region, the following condition holds
$$\begin{align}
  v_{GS} &\gt V_{th}\\
  v_{DS} &\gt v_{GS} - V_{th}
\end{align}\tag{3}$$
where $V_{th}$ is the gate-source threshold voltage.

In active region, the drain current $i_D$ is independent from the drain-source voltage
$$ i_D =  K(v_{GS}-V_{th})^2 \tag{4}$$
where $K$ is a constant determined by the physical parameters of the device.

#Boundary between the Ohmic and Active Regions

On the boundary between the ohmic region and the active region, the following condition holds
$$\begin{align}
  v_{GS} &\gt V_{th}\tag{5}\\
  v_{DS} &= v_{GS} - V_{th}\tag{6}
\end{align}$$
where $V_{th}$ is the gate-source threshold voltage.

Substituting (6) into (2) or (4) yields the drain current $i_D$ determined by
$$i_D =  Kv_{DS}^2 \tag{7}$$
where $K$ is a constant determined by the physical parameters of the device.

#Python Code
The Python code illustrates the VI characteristics of power MOSFETs.

In [ ]:
import matplotlib
from os import linesep
from numpy import linspace,logspace,exp,log,fft,pi,arccos,cos,heaviside
import matplotlib.pyplot as plt
from scipy.integrate import quad
from matplotlib import animation,rc
rc('animation', html='jshtml')
plt.rcParams.update({"font.family" : "sans serif","mathtext.fontset" : "cm"})

def iD(vGS,Vth,vDS,K): #ohmic and active regions
  ids = heaviside(vGS-Vth,1)*(heaviside(vDS-(vGS-Vth),1)*K*(vGS - Vth)**2 +
                             heaviside((vGS-Vth)-vDS,0)*K*(2*(vGS - Vth)*vDS - vDS**2))
  return ids
def iD_boundary(vDS,K): #boundary between ohmic and active regions
  return K*vDS**2

#parameters
clst=['lightskyblue','violet','aqua','cornflowerblue','yellow','w',(.1,.1,.1)] #colors dark bg
Nf = 100
Vth = 4
K=1 #determined by phsysical parameters such as material and device geometry
vDS_max = 25
vDS=linspace(0,vDS_max,Nf)
vGS = linspace(7,15,9)
vDS_b = linspace(0,vGS[-1]*1.05-Vth,Nf) #for boundary plotting
iD_lst=[]
for v in vGS:
  iD_lst.append(iD(v,Vth,vDS,K))
iD_max = max(iD_lst[-1])
iD_b = iD_boundary(vDS_b,K) #boundary between active and ohmic regions

#set up figures
fig = plt.figure(figsize=(9,9),facecolor=clst[-1])
ax_frame = [[0, 0, .95, .95], [-.1,1.1], [-.1,1.1]]# [pos-boundary, xlim, ylim]
xn,xm,yn,ym = ax_frame[1]+ax_frame[2]
ax=fig.add_axes(ax_frame[0],xlim=ax_frame[1], ylim=ax_frame[2],fc='none',) #no face color
ax.axis('off')
ax.annotate("", (xm, 0), (xn, 0),arrowprops={'arrowstyle':"->",'color':clst[-2]}) #x-axis
ax.annotate("", (0,ym), (0,yn),arrowprops={'arrowstyle':"->",'color':clst[-2]}) #y-axis
ax.text(xm-.02,0,r'$v_{DS}$',size=24,va='top',ha='right',color=clst[-2]) #x-axis label
ax.text(.02,ym,r'$i_{D}$',size=24,va='top',color=clst[-2]) #x-axis label
lines_iD = [] #lines for the family of iD-vDS curves
for i in range(len(vGS)):
  lines_iD.append(ax.plot([],[],lw=3,color=clst[i%4])[0])
line_boundary = ax.plot([],[],lw=3,ls='--',color=clst[4])[0]
# animation function. This is called sequentially
def animate(i):
  ii = i%Nf
  k = int(i/Nf)
  if ii==0:
    print(k)
  if k < len(vGS):
    lines_iD[k].set_data(vDS[:ii+1]/vDS_max,iD_lst[k][:ii+1]/iD_max)
    if ii==Nf-1:
      ax.text(1,iD_lst[k][-1]/iD_max,'$v_{GS} = '+str(vGS[k])+'\ V$ ',color=clst[-2],
              size=20,ha='right', va='center',backgroundcolor=(.2,.2,.2))
  else:
    line_boundary.set_data(vDS_b[:ii+1]/vDS_max,iD_b[:ii+1]/iD_max)
  return
anim = animation.FuncAnimation(fig, animate, frames=Nf*(len(vGS)+1), interval=20)
# anim #uncomment to generate animation in the output area
anim.save("MOSFET_VIcurves.mp4", fps=35, dpi = 120) #uncomment to save